# Copyright 2022 Cognite AS

## Import the Libraries and Modules

In [12]:
import os
import sys

In [13]:
module_path = os.path.abspath(os.path.join("../utils"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/cram/dev/using-cognite-python-sdk/utils


In [14]:
import auth as cauth

## Create the Cognite Client

In [15]:
c = cauth.create_cognite_client(method="client-secret")

/home/cram/dev/using-cognite-python-sdk/.venv/lib/python3.8/site-packages/cognite/client/_cognite_client.py:85: UserWarning: You are using version 2.56.0 of the SDK, however version 2.56.1 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  self._config = ClientConfig(


## Delete the data in CDF

### Delete using ids

In [17]:
c.sequences.data.retrieve_dataframe(id=2357839693666680,start=0,end=None)

,user,amount
1,user1,3.14
2,user2,2.72


In [18]:
c.sequences.data.delete(id=2357839693666680, rows=[1])

In [19]:
c.sequences.data.retrieve_dataframe(id=2357839693666680,start=0,end=None)

,user,amount
2,user2,2.72


### Delete using a Range

In [10]:
c.time_series.list(external_id_prefix="my_")

,id,external_id,name,is_string,metadata,is_step,security_categories,data_set_id,created_time,last_updated_time
0,5258405260776053,my_ts,my_ts,False,{},False,[],1399248851345249,1657199813970,1657199813970


In [11]:
c.datapoints.delete_range(start=150000000000, end=160000000000, id=5258405260776053)

In [20]:
c.sequences.data.delete_range(id=2357839693666680, start=0, end=None)

In [21]:
c.sequences.data.retrieve_dataframe(id=2357839693666680,start=0,end=None)

,user,amount


### Delete multiple ranges

In [ ]:
ranges = [{"id": 1, "start": "2d-ago", "end": "now"},
            {"externalId": "abc", "start": "2d-ago", "end": "now"}]
c.datapoints.delete_ranges(ranges)

### Delete various resource types

In [24]:
c.labels.list()

,external_id,name,created_time
0,REGION,REGION,1657141671698
1,COUNTRY,COUNTRY,1657141671698
2,BALTIC,BALTIC,1657143421635
3,NORDIC,NORDIC,1657142354433


In [23]:
c.labels.delete(external_id=["ROTATING_EQUIPMENT", "PUMP"])

In [9]:
c.assets.delete(id=[2337497105272481,3850461732700104])

In [ ]:
c.assets.delete(id=[1,2,3], external_id="3")

In [ ]:
c.time_series.delete(id=[1,2,3], external_id="3")

In [ ]:
c.sequences.delete(id=[1,2,3], external_id="3")

In [ ]:
c.events.delete(id=[1,2,3], external_id="3")

In [ ]:
c.files.delete(id=[1,2,3], external_id="3")

In [ ]:
c.relationships.delete(external_id=["a","b"])

## Bulk Deletion

How to delete recursively and effectively.

In [25]:
c.assets.retrieve_subtree(external_id="root")

,external_id,name,data_set_id,metadata,id,created_time,last_updated_time,root_id,parent_id,parent_external_id
0,root,Root,1399248851345249,{},4299328479638485,1657200692566,1657200692566,4299328479638485,<NA>,NaN
1,child2,child2,1399248851345249,{},2994057977739744,1657200692566,1657200692566,4299328479638485,4299328479638485,root
2,child1,child1,1399248851345249,{},6709553116468444,1657200692566,1657200692566,4299328479638485,4299328479638485,root
3,child_of_child1,child_of_child1,1399248851345249,{},6986429626685430,1657200692566,1657200692566,4299328479638485,6709553116468444,child1


In [26]:
# Retrieve a subtree given a root asset and delete root and all subtree assets
empty_asset_list = False
root_asset_ext_id = 'root'
while not(empty_asset_list):
    try:
        subtree_df = c.assets.retrieve(external_id=root_asset_ext_id).subtree().to_pandas()
        non_parent_childs = subtree_df.loc[~subtree_df['id'].isin(subtree_df['parentId'].unique()),:]['id'].tolist()
        c.assets.delete(non_parent_childs)
        empty_asset_list = False
    except:
        empty_asset_list = True
        try:
            c.assets.delete(external_id=root_asset_ext_id)
        except:
            pass

In [27]:
c.assets.list(data_set_external_ids=['root'])

""
